<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/project-name.png" align = "left" width = "70%" />

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/team-name.png" width = "40%" align = "left" />

<img src="https://images.unsplash.com/photo-1569060368645-4ab30c8d8b0e?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1920" width = "60%" align = "left" />

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/overview.png" width = "30%" align = "left" />

1. Introduction
2. Import Packages and Data
3. Data Exploration
4. Data Preprocessing
5. Model Building
6. Model Prediction
7. Conclusion
<br>

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/introduction.png" width = "40%" align = "left" />

## Problem Defintion

## Problem Statement
To determine the belief in climate change threat by building a (machine learning) classifier and performing sentiment analysis based off tweets to assist market researchers gauge how their products may be received by the consumers.

#### Why
This will give companies built around lessening one’s environmental impact or carbon footprint access to a broad base of consumer sentiment, spanning multiple demographic and geographic categories - thus increasing their insights and informing future marketing strategies.

#### How
By Analysing text data from tweets posted about climate change and predicting the sentiment that people have about climate change

#### What
A Machine Learning model deployed on a web application that is able to classify their sentiment about climate change, based on their novel tweet data

## Problem Landscape


### Data
* Open-source Tweet Data
* Data from twitter made available on Kaggle
* structured

#### Information
1. Classification Machine Learning
2. Climate Change


#### Knowledge
[... something... ]


## Equation of Value


$NC = $New Clients + Customer Churn ($v$)

$Ppp = $ Price per Product

$Score_{ML} = $ A measure of the machine learning model's predictive ability - typically (F1 score, accuracy or precision from the _Confusion Matrix_). For this project the **F1 Score** will be used.


$$ NC = v(Score_{ML})$$

$$ Revenue = f(Ppp, NC) $$


The machine learning model can add value by helping companies identify which areas will lead to more sales.




# Machine Learning

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/import-packages.png" width = "30%" align = "left" />

In [ ]:
# Add the following code anywhere in your machine learning file
experiment = comet_ml.Experiment(api_key="quY9CXKJTLd4wCLNuIQqCuVGa",
                        project_name="joziradicals-jhb-ss5-classification",
                        workspace="tiroamodimo")

In [1]:
!pip install comet_ml
# !pip install nltk
# !pip install spacy
# !pip install textblob


     |████████████████████████████████| 214 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 8.3 MB/s eta 0:00:01
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_64.whl size=37295 sha256=0f8b1080b7b1a6108b4f854e96e807bde7072bd88845f38df95144a5b996be90
  Stored in directory: /root/.cache/pip/wheels/5f/09/cf/2b1aa8371c071fa89518ac0bbda1b8cca4e65b6e2538af4192
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34546 sha256=1115f1e61f3ce7d2b739cb856125c7c6a07292ff1736aeb613f66f9945984902
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built netifaces configobj


In [1]:
# data analysis libraries
import numpy as np
import pandas as pd
import string
import re
import string
import scipy.sparse

# other
import os
import warnings

# visualisation libraries
from matplotlib import pyplot as plt
import seaborn as sns

# Notebook styling
%matplotlib
sns.set()
warnings.filterwarnings("ignore")

# NLP
import nltk

from nltk.corpus import stopwords
import spacy
#import textblob
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# ML Pre processing
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# ML Model Building
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


# ML Model Testing and Optimisation
from sklearn.utils import resample
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# ML Model Deployment
import pickle

Using matplotlib backend: Qt5Agg


<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/import-data.png" width = "20%" align = "left" />

In [2]:
# check data files
basepath = 'resources/data'

In [3]:
# import train and test data files
train_df = pd.read_csv(basepath + '/train.csv')
test_df = pd.read_csv(basepath + '/test.csv')
#sample_submission_df = pd.read_csv(basepath + '/sample_submission.csv')

In [4]:
# check training data
train_df.head()

,sentiment,message,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553
1,-1,All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504


In [5]:
# check test data
test_df.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/eda-header.png" width = "70%" align = "left" />

After the data cleaning step where we put our data into a few standard formats, the next step is to take a look at the data and see if what we're looking at makes sense.

**Analysis of Words**

* Word Count - Total Number of Words
* Word Frequency  - find most common and create word clouds
* Uncommon Word Count - number of uncommon words
* Uncommon Word Frequnce - find least common and create word clouds
* Parts of Speech - 8 parts of speech will be analysed

**Analysis of Twitter Indicators**
* Use of handles
    * How many use handles
    * Use of frequently used handles - bar chart

In [6]:
# view training data
train_df.head()

,sentiment,message,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553
1,-1,All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504


In [7]:
# view test data
test_df.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [8]:
# Check number of rows and columns
print('Number of rows and columns in train data: {}'.format(train_df.shape))
print('Number of rows and columns in test data: {}'.format(test_df.shape))

Number of rows and columns in train data: (30759, 3)
Number of rows and columns in test data: (10546, 2)


### Check for missing data in any of the columns

In [9]:
train_df.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [10]:
test_df.isnull().sum()

message    0
tweetid    0
dtype: int64

### How many posts are there for each sentiment?

In [11]:
post_sum = train_df.groupby(['sentiment']).count()
post_sum.sort_values('message', ascending=False, inplace=True)
post_sum = post_sum.drop(["tweetid"], axis=1)
post_sum

,message
sentiment,
1,16073
2,6493
0,5400
-1,2793


The positive sentiments have the most posts combined. <br>

<br>

The sentiment that has the least posts is '-1', it <br> 
seems that most people believe in climate change. 

### What are the total words written by each sentiment type?

In [12]:
train_df['word_count'] = train_df['message'].apply(lambda x: len(str(x).split(" ")))
word_count = train_df.groupby('sentiment').sum()
word_count = word_count.drop(["tweetid"], axis=1)
word_count.sort_values('word_count', ascending=False, inplace=True)
word_count

,word_count
sentiment,
1,295413.0
2,99547.0
0,87417.0
-1,50561.0


Not surprisingly, the most words are written by the posts <br>
with a positive sentiment. <br>

<br>

The sentiment that had the least words was '-1', <br> 
and follows the trend above.

### Distribution of the Sentiment Values

In [13]:
# Setup chart size
dim = (15.0, 4.0)
fig, ax = plt.subplots(figsize=dim)


# Create color palette
cmrmap = sns.color_palette('YlGn')
sns.set_palette(cmrmap)

# Connect data to chart
sns.countplot(x='sentiment', data=train_df, order=[-1, 0, 1, 2])

# Create labels
plt.title('Distribution of Sentiments in the Dataset', fontsize=16)
plt.xlabel('Sentiment Value')
plt.ylabel('Count of Posts')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

The "Anti" sentiment has the least posts, <br>
whilst the "Pro" sentiment has the most posts. <br>

<br>

This indicates that the data is imbalanced, <br>
and we will need to apply sampling techniques.

In [14]:
# code for wordclouds goes between here and Sentiment Analysis section

## Sentiment Analysis

We made use of Vader to do sentiment analysis on each tweet.<br>
VADER belongs to a type of sentiment analysis that is based on lexicons of sentiment-related words.<br>
In this approach, each of the words in the lexicon is rated as to whether it is positive or negative, and in many cases, how positive or negative.

In [15]:
# instantiate SentimentIntensityAnalyzer object
sid = SentimentIntensityAnalyzer()

# Creates a new column called scores
train_df['scores'] = train_df['message'].apply(lambda message: sid.polarity_scores(message))
test_df['scores'] = test_df['message'].apply(lambda message: sid.polarity_scores(message))


# Creates a new column called compound that contains the compound score of each tweet
train_df['compound']  = train_df['scores'].apply(lambda score_dict: score_dict['compound'])
test_df['compound']  = test_df['scores'].apply(lambda score_dict: score_dict['compound'])

Let's take a look now

In [16]:
train_df.head()

,sentiment,message,tweetid,word_count,scores,compound
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,21,"{'neg': 0.19, 'neu': 0.698, 'pos': 0.112, 'com...",-0.4574
1,-1,All the biggest lies about climate change and ...,925046776553529344,16,"{'neg': 0.152, 'neu': 0.761, 'pos': 0.087, 'co...",-0.2960
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,18,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,21,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,19,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou...",-0.7430


In [17]:
test_df.head()

,message,tweetid,scores,compound
0,Europe will now be looking to China to make su...,169760,"{'neg': 0.0, 'neu': 0.734, 'pos': 0.266, 'comp...",0.6310
1,Combine this with the polling of staffers re c...,35326,"{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'comp...",-0.5574
2,"The scary, unimpeachable evidence that climate...",224985,"{'neg': 0.198, 'neu': 0.802, 'pos': 0.0, 'comp...",-0.4939
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,"{'neg': 0.107, 'neu': 0.893, 'pos': 0.0, 'comp...",-0.3382
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,"{'neg': 0.223, 'neu': 0.777, 'pos': 0.0, 'comp...",-0.3164


We now assign the sentiment 'Postive', 'Negative or 'Neutral' based on the compound score of the tweet. 

In [18]:
# create list of compound scores in train dataset
compound = list(train_df['compound'])

# instantiate empty list
com =[]

# assign 'Postive', 'Negative or 'Neutral' based on the compound score of the tweet.
for i in compound:
    if i >= 0.05:
        com.append(' .Positive.')
    elif i > -0.05 and i <0.05:
        com.append(' .Neutral.')
    elif i <= -0.05 : #or else
        com.append(' .Negative.')

# update training set dataframe
train_df['calculated_sent'] = com

# create list of compound scores in test set dataframe
compound = list(test_df['compound'])

# instantiate empty list
com =[]

# assign 'Postive', 'Negative or 'Neutral' based on the compound score of the tweet.
for i in compound:
    if i >= 0.05:
        com.append(' .Positive.')
    elif i > -0.05 and i <0.05:
        com.append(' .Neutral.')
    elif i <= -0.05 : #or else
        com.append(' .Negative.')
        
# update training set dataframe
test_df['calculated_sent'] = com

Let's take a look now

In [19]:
train_df.head()

,sentiment,message,tweetid,word_count,scores,compound,calculated_sent
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,21,"{'neg': 0.19, 'neu': 0.698, 'pos': 0.112, 'com...",-0.4574,.Negative.
1,-1,All the biggest lies about climate change and ...,925046776553529344,16,"{'neg': 0.152, 'neu': 0.761, 'pos': 0.087, 'co...",-0.2960,.Negative.
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,18,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,.Neutral.
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,21,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,.Neutral.
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,19,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou...",-0.7430,.Negative.


In [20]:
test_df.head()

,message,tweetid,scores,compound,calculated_sent
0,Europe will now be looking to China to make su...,169760,"{'neg': 0.0, 'neu': 0.734, 'pos': 0.266, 'comp...",0.6310,.Positive.
1,Combine this with the polling of staffers re c...,35326,"{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'comp...",-0.5574,.Negative.
2,"The scary, unimpeachable evidence that climate...",224985,"{'neg': 0.198, 'neu': 0.802, 'pos': 0.0, 'comp...",-0.4939,.Negative.
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,"{'neg': 0.107, 'neu': 0.893, 'pos': 0.0, 'comp...",-0.3382,.Negative.
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,"{'neg': 0.223, 'neu': 0.777, 'pos': 0.0, 'comp...",-0.3164,.Negative.


We add the sentiment to the end of the message.

In [21]:
train_df['message'] = train_df['message'].astype(str).add(train_df['calculated_sent'])
test_df['message'] = test_df['message'].astype(str).add(test_df['calculated_sent'])

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/data-preprocessing.png" width = "60%" align = "left" />

### Functions for Data Cleaning

For the cleaning of our data we will be making use of multiple functions. Some of which we found on the internet and other kaggle notebooks.

In [22]:

def make_lower(text):
    
    """
    This function takes a string of text as input. It makes the strings lowercase and returns the modified text.
    """
    
    # return lowercase
    return text.lower()


def clean_url(text):
    
    """
    This function takes a string of text as input. It replaces the url with with the string 'web-url
    from the text string and returns the modified text string.
    """
    
    # define RegEx pattern for a url link
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    
    # define replacement word
    subs_url = r'web-url'
    
    # replace urls with 'web-url'
    return re.sub(pattern_url, subs_url, text)
    
    
def remove_punctuation_numbers(text):
    
    """
    This function takes a string of text as input. It removes punctuation and numbers from the text string
    and returns the modified text string.
    """
    
    # define punctuation and numbers
    punc_numbers = string.punctuation + '0123456789'
    
    
    # return the modeified text
    return ' '.join([l for l in text.split() if l not in punc_numbers])


def clean(tweet):
    
    """
    This function takes some words that are contracted and expands them
    """
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    
    return tweet


def correct_spelling(text):
    
    """
    This function takes a string of text as input. It corrects the spelling by applying textblob's correction
    method and returns the modified text string.
    """
    
    # instantiate TextBlob object
    blob = TextBlob(text)
    
    # correct spelling and return modified string
    return str(blob.correct())
    



In [23]:
## Clean urls
print ('Cleaning URLs...')

train_df['message'] = train_df['message'].apply(clean_url)
test_df['message'] = test_df['message'].apply(clean_url)

# Remove punctuation
print ('Removing punctuation...')
train_df['message'] = train_df['message'].apply(remove_punctuation_numbers)
test_df['message'] = test_df['message'].apply(remove_punctuation_numbers)


# Make lower case
print ('Lowering case...')

train_df['message'] = train_df['message'].apply(make_lower)
test_df['message'] = test_df['message'].apply(make_lower)


## Remove rt
print ('Removing rt...')

train_df['message'] = train_df['message'].replace(to_replace = r'rt', value = '', regex = True)
test_df['message'] = test_df['message'].replace(to_replace = r'rt', value = '', regex = True)

##cleaning function
print('Expanding words...')
train_df['message'] = train_df['message'].apply(clean)
test_df['message'] = test_df['message'].apply(clean)

# correct spelling
#print ('Removing punctuation...')
#train_df['message'] = train_df['message'].apply(correct_spelling)

Cleaning URLs...
Removing punctuation...
Lowering case...
Removing rt...
Expanding words...


## Stemming and Lemmatization

Stemming and Lemmatization are Text Normalization (or sometimes called Word Normalization) techniques in the field of Natural Language Processing that are used to prepare text, words, and documents for further processing.

After testing both stemming and lemmatization on our text we discovered that using lemmatization has better results 
than when stemming is used.

In [24]:
# Applying lemmatization to the train and test message data
lemmatizer = WordNetLemmatizer()
train_df['lemma'] = [' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])for text in train_df['message']]
test_df['lemma'] = [' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])for text in test_df['message']]

## Word Vectors (Feature Engineering)

We will be using Count Vectorizer as well as  Tfidf Vectorizer. <br>
We decided to use the full set of features(tfidf+counts). 

In [25]:
X = train_df['lemma']
y = train_df['sentiment']
X_test = test_df['lemma']


In [26]:
vectorizer=TfidfVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
vectorized=vectorizer.fit_transform(X)

count_vectorizer=CountVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
count_vectorized=count_vectorizer.fit_transform(X)
X = scipy.sparse.hstack([vectorized, count_vectorized])

In [27]:
#vectorizer=TfidfVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
vectorized=vectorizer.transform(X_test)

#count_vectorizer=CountVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
count_vectorized=count_vectorizer.transform(X_test)

X_test = scipy.sparse.hstack([vectorized, count_vectorized])

## Data Test Train Split

In [36]:
#Splitting our train data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                    y,
                                                    test_size =0.2,
                                                   random_state = 42) 

In [28]:
X_train = X
y_train = y

## Classification Experiments

1. Connect Model Experiment to comet
2. Train Model
3. Test and Evaluate Model
4. Deploy Model

    4.1 Deploy in Kaggle
    
    4.2 Deploy in Comet

### Functions for Functions for Model Evaluation and Deployment

In [29]:
def deploy_comet(experiment, metrics, parameters=None):
    """
    This function takes a comet experiment object, a dictionary of model parameters
    and a dictionary of model test results as inputs and uploads the experiment to comet.
    """
    
    # Log our parameters
    if parameters != None:
        print('logging parameters...')
        experiment.log_parameters(params)
    
    # log model performace
    print('logging metric...')
    experiment.log_metrics(metrics)
    
    print('ending experiment...')
    # end experiment
    experiment.end()  

    
    
def dict_to_df(dict_data):
    """
    This function take a dictionary of data and returns a dataframe of the data.
    """
    
    # convert dictionary to dataframe
    return pd.DataFrame(dict_data, index=['Value']).T



def make_submission_csv(model_name, clf, features):
    
    """
    This funtion this model take a file name as string, model object and vector of features
    as inputs and returns a csv to submit to Kaggle or Zindi
    """
    
    # define columns in sample submission data
    cols = sample_submission_df.columns
    
    # Make prediction of submission data
    print('making prediction...')
    prediction = clf.predict(features)
    
    # make dataframe of predictions of submission data
    print('creating dataframe... ')
    submission = pd.DataFrame({cols[0]: sample_submission_df[cols[0]],
                               cols[1]: prediction})
    
    # save file
    filename = 'submission_' + model_name + '.csv'
    submission.to_csv(filename,index=False)
    print(f'Saved file {filename}')


def make_pickle(model_name, model):
    
    """
    This function takes the string of a model name and model object as input and
    saves a pickle file of the model in the current directory of the file.
    """
    
    # create file name
    model_save_path = model_name + ".pkl"

    # open file to write binary
    with open(model_save_path,'wb') as file:

        # save model as file name
        pickle.dump(model,file)
        print(f'Pickle file for {model_name} successfully saved')

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/model-building.png" width = "50%" align = "left" />

#### Base 1 Linear SVC

In [66]:
clf =   LinearSVC().fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [67]:
print(metrics.confusion_matrix(y_val,y_pred))

[[ 145   35   83   15]
 [  20  198  174   33]
 [  42  130 1420  163]
 [   6   18  131  551]]


In [68]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.68      0.52      0.59       278
           0       0.52      0.47      0.49       425
           1       0.79      0.81      0.80      1755
           2       0.72      0.78      0.75       706

    accuracy                           0.73      3164
   macro avg       0.68      0.64      0.66      3164
weighted avg       0.73      0.73      0.73      3164



In [69]:
# Evaluate Model Performace
accuracy_base_lsvc = metrics.accuracy_score(y_val,y_pred)
precision_base_lsvc = metrics.precision_score(y_val,y_pred, average ='weighted')
recall_base_lsvc = metrics.recall_score(y_val,y_pred, average ='weighted')
f1_base_lsvc = metrics.f1_score(y_val,y_pred, average ='weighted')

# save model metrics
metrics_base_lsvc = {'accuracy': accuracy_base_lsvc,
                     'precision': precision_base_lsvc,
                     'recall': recall_base_lsvc,
                     'f1_score': f1_base_lsvc}

# show model performance
dict_to_df(metrics_base_lsvc)

,Value
accuracy,0.731353
precision,0.726610
recall,0.731353
f1_score,0.727517


In [70]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('base_lsvc', clf, X_test)

# save pickle file
make_pickle('base_lsvc', clf)

making prediction...
creating dataframe... 
Saved file submission_base_lsvc.csv
Pickle file for base_lsvc successfully saved


#### Base 2 Naive Bayes

In [71]:
clf =  MultinomialNB().fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [72]:
print(metrics.confusion_matrix(y_val,y_pred))

[[  51   20  179   28]
 [   8  116  255   46]
 [   2   28 1553  172]
 [   1    4  171  530]]


In [73]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.82      0.18      0.30       278
           0       0.69      0.27      0.39       425
           1       0.72      0.88      0.79      1755
           2       0.68      0.75      0.72       706

    accuracy                           0.71      3164
   macro avg       0.73      0.52      0.55      3164
weighted avg       0.72      0.71      0.68      3164



In [74]:
# Evaluate Model Performace
accuracy_nb = metrics.accuracy_score(y_val,y_pred)
precision_nb = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_nb = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_nb = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save model metrics
metrics_base_nb = {'accuracy': accuracy_nb,
                     'precision': precision_nb,
                     'recall': recall_nb,
                     'f1_score': f1_nb}

# show model performance
dict_to_df(metrics_base_nb)

,Value
accuracy,0.711125
precision,0.716594
recall,0.711125
f1_score,0.678791


In [75]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('base_mnb', clf, X_test)

# save pickle file
make_pickle('base_mnb', clf)

making prediction...
creating dataframe... 
Saved file submission_base_mnb.csv
Pickle file for base_mnb successfully saved


#### Base 3 Logistic Regression

In [76]:
clf =  LogisticRegression().fit(X_train, y_train)
y_pred = clf.predict(X_val)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [77]:
print(metrics.confusion_matrix(y_val,y_pred))

[[ 131   36  100   11]
 [  17  198  186   24]
 [  26  100 1502  127]
 [   5   19  130  552]]


In [78]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.47      0.57       278
           0       0.56      0.47      0.51       425
           1       0.78      0.86      0.82      1755
           2       0.77      0.78      0.78       706

    accuracy                           0.75      3164
   macro avg       0.71      0.64      0.67      3164
weighted avg       0.75      0.75      0.75      3164



In [79]:
# Evaluate Model Performace
accuracy_log = metrics.accuracy_score(y_val,y_pred)
precision_log = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_log = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_log = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save model metrics
metrics_base_lr = {'accuracy': accuracy_log,
                     'precision': precision_log,
                     'recall': recall_log,
                     'f1_score': f1_log}

# show model performance
dict_to_df(metrics_base_lr)

,Value
accuracy,0.753161
precision,0.746525
recall,0.753161
f1_score,0.745871


In [80]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('base_lr', clf, X_test)

# save pickle file
make_pickle('base_lr', clf)

making prediction...
creating dataframe... 
Saved file submission_base_lr.csv
Pickle file for base_lr successfully saved


#### Base 4 K-Nearest Neighbour

In [82]:
clf = KNeighborsClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [84]:
print(metrics.confusion_matrix(y_val,y_pred))

[[  49  133   94    2]
 [  16  309   98    2]
 [  39  653 1006   57]
 [   3  201  248  254]]


In [85]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.46      0.18      0.25       278
           0       0.24      0.73      0.36       425
           1       0.70      0.57      0.63      1755
           2       0.81      0.36      0.50       706

    accuracy                           0.51      3164
   macro avg       0.55      0.46      0.43      3164
weighted avg       0.64      0.51      0.53      3164



In [87]:
accuracy_kn = metrics.accuracy_score(y_val,y_pred)
precision_kn = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_kn = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_kn = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save model metrics
metrics_base_knn = {'accuracy': accuracy_kn,
                     'precision': precision_kn,
                     'recall': recall_kn,
                     'f1_score': f1_kn}

# show model performance
dict_to_df(metrics_base_knn)

,Value
accuracy,0.511378
precision,0.638084
recall,0.511378
f1_score,0.530266


In [88]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('base_knn', clf, X_test)

# save pickle file
make_pickle('base_knn', clf)

making prediction...
creating dataframe... 
Saved file submission_base_knn.csv
Pickle file for base_knn successfully saved


#### Base 6 Random Forest

In [89]:
clf = RandomForestClassifier(class_weight = "balanced").fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [91]:
print(metrics.confusion_matrix(y_val,y_pred))

[[  62   32  171   13]
 [   3  175  214   33]
 [   4   97 1471  183]
 [   0   13  182  511]]


In [92]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.90      0.22      0.36       278
           0       0.55      0.41      0.47       425
           1       0.72      0.84      0.78      1755
           2       0.69      0.72      0.71       706

    accuracy                           0.70      3164
   macro avg       0.72      0.55      0.58      3164
weighted avg       0.71      0.70      0.68      3164



In [93]:
accuracy_rf = metrics.accuracy_score(y_val,y_pred)
precision_rf = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_rf = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_rf = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save model metrics
metrics_base_rf = {'accuracy': accuracy_rf,
                     'precision': precision_rf,
                     'recall': recall_rf,
                     'f1_score': f1_rf}

# show model performance
dict_to_df(metrics_base_rf)

,Value
accuracy,0.701327
precision,0.707546
recall,0.701327
f1_score,0.682695


In [94]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('base_rf', clf, X_test)

# save pickle file
make_pickle('base_rf', clf)

making prediction...
creating dataframe... 
Saved file submission_base_rf.csv
Pickle file for base_rf successfully saved


#### Base 7 Neural Network

Comparing Base Models

In [ ]:
accuracy = [accuracy_log,accuracy_sv, accuracy_nb, accuracy_kn, accuracy_rf]
f1_score = [f1_log, f1_sv, f1_nb, f1_kn, f1_rf]
results_base = pd.DataFrame()



results_base['F1_base'] = f1_score
results_base['Accuracy_base'] = accuracy
results_base['models'] = ['Logistic Regression', 'Linear Support Vector', 'Naive Bayes', 'K Neighbours', 'Random Forest']
results_base.set_index('models', inplace= True)

### Tuned Models

#### Tuned 1 Linear SVC

In [95]:
def tune_LSVC_model(X_train, y_train): 
    C_list = [0.001, 0.01, 0.1, 0.5, 0.75, 1, 5, 10, 25, 100]
    penalty_list = ['l1','l2']
    loss =['hinge', 'squared_hinge']
   
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')


    
    lsvc = LinearSVC()
    
    parameters = {'C':C_list,
                  'penalty': penalty_list,
                  'loss' : loss
                     }
    tune = GridSearchCV(lsvc, parameters, scoring = scorer)
    tune.fit(X_train,y_train)
    
    return tune

In [98]:
clf = tune_LSVC_model(X_train, y_train)
y_pred = clf.predict(X_val)

In [99]:
# Confusion Matrix
print(metrics.confusion_matrix(y_val,y_pred))

[[ 128   38   99   13]
 [  13  194  185   33]
 [  19   92 1500  144]
 [   6    9  132  559]]


In [100]:
# Classification Report
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.77      0.46      0.58       278
           0       0.58      0.46      0.51       425
           1       0.78      0.85      0.82      1755
           2       0.75      0.79      0.77       706

    accuracy                           0.75      3164
   macro avg       0.72      0.64      0.67      3164
weighted avg       0.75      0.75      0.74      3164



In [102]:
# Evaluate model performanace
accuracy_sv = metrics.accuracy_score(y_val,y_pred)
precision_sv = metrics.precision_score(y_val,y_pred, average ='weighted')
recall_sv = metrics.recall_score(y_val,y_pred, average ='weighted')
f1_sv = metrics.f1_score(y_val,y_pred, average ='weighted')

# save performance metrics
metrics_tuned_lsvc = {'accuracy': accuracy_sv,
                     'precision': precision_sv,
                     'recall': recall_sv,
                     'f1_score': f1_sv}

# show model performance
dict_to_df(metrics_tuned_lsvc)

,Value
accuracy,0.752528
precision,0.746784
recall,0.752528
f1_score,0.744162


In [103]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('tuned_lsvc', clf, X_test)

# save pickle file
make_pickle('tuned_lsvc', clf)

making prediction...
creating dataframe... 
Saved file submission_tuned_lsvc.csv
Pickle file for tuned_lsvc successfully saved


#### Adjusted 2 Naive Bayes

In [36]:
def tune_NB_model(X_train, y_train): 
    alpha = ['alpha', 1e-10, 1]
    fit_prior = ['fit_prior', [True, False]]
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')


    
    nb = MultinomialNB()
    
    parameters = {'alpha': alpha,
                  'fit_prior': fit_prior
                     }
    tune = GridSearchCV(nb, parameters, scoring = scorer)
    tune.fit(X_train,y_train)
    
    return tune

In [37]:
clf = tune_NB_model(X_train, y_train)
#y_pred = tune_nb.predict(X_val)

In [109]:
# Confusion Matrix
print(metrics.confusion_matrix(y_val,y_pred))

[[ 137   35   91   15]
 [  63  168  141   53]
 [  96  176 1261  222]
 [  11   30  159  506]]


In [110]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.45      0.49      0.47       278
           0       0.41      0.40      0.40       425
           1       0.76      0.72      0.74      1755
           2       0.64      0.72      0.67       706

    accuracy                           0.65      3164
   macro avg       0.56      0.58      0.57      3164
weighted avg       0.66      0.65      0.66      3164



In [112]:
# Evaluate Model Performance
accuracy_nb = metrics.accuracy_score(y_val,y_pred)
precision_nb = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_nb = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_nb = metrics.f1_score(y_val,y_pred, average = 'weighted')


# save performance metrics
metrics_tuned_nb = {'accuracy': accuracy_nb,
                     'precision': precision_nb,
                     'recall': recall_nb,
                     'f1_score': f1_nb}

# show model performance
dict_to_df(metrics_tuned_nb)

,Value
accuracy,0.654867
precision,0.659621
recall,0.654867
f1_score,0.656206


In [113]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('tuned_naive_beyes', clf, X_test)

# save pickle file
make_pickle('tuned_naive_beyes', clf)

making prediction...
creating dataframe... 
Saved file submission_tuned_naive_beyes.csv
Pickle file for tuned_naive_beyes successfully saved


 #### Adjusted 3 Logistic Regression

In [30]:
def tune_LogReg_model(X_train, y_train): 
    C_list = [0.001, 0.01, 0.1, 0.5, 0.75, 1, 5, 10, 25, 100]
    penalty_list = ['l1','l2']
    random_state = ['random_state', 1, 10]
    tol = ['tol', 1e-10, 1]
    max_iter = ['max_iter', 1000, 10000]
    warm_start = ['warm_start', [True, False]]
   
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')


    
    logreg = LogisticRegression()
    
    parameters = {'C':C_list,
                  'penalty': penalty_list,
                  'random_state' : random_state,
                  'tol': tol,
                  
                     }
    tune = GridSearchCV(logreg, parameters, scoring = scorer)
    tune.fit(X_train,y_train)
    
    return tune

In [31]:
clf = tune_LogReg_model(X_train, y_train)
y_pred = clf.predict(X_val)

NameError: name 'X_val' is not defined

In [33]:
clf.best_estimator_

LogisticRegression(C=0.75, random_state=1, tol=1e-10)

In [121]:
# Confusion Matrix
print(metrics.confusion_matrix(y_val,y_pred))

[[ 131   36  100   11]
 [  17  198  186   24]
 [  26  100 1502  127]
 [   5   19  130  552]]


In [122]:
# Classification Report
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.47      0.57       278
           0       0.56      0.47      0.51       425
           1       0.78      0.86      0.82      1755
           2       0.77      0.78      0.78       706

    accuracy                           0.75      3164
   macro avg       0.71      0.64      0.67      3164
weighted avg       0.75      0.75      0.75      3164



In [123]:
accuracy_log = metrics.accuracy_score(y_val,y_pred)
precision_log = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_log = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_log = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save performance metrics
metrics_tuned_lr = {'accuracy': accuracy_log,
                     'precision': precision_log,
                     'recall': recall_log,
                     'f1_score': f1_log}

# show model performance
dict_to_df(metrics_tuned_lr)

,Value
accuracy,0.753161
precision,0.746525
recall,0.753161
f1_score,0.745871


In [124]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('tuned_logistic_regression', clf, X_test)

# save pickle file
make_pickle('tuned_logistic_regression', clf)

making prediction...
creating dataframe... 
Saved file submission_tuned_logistic_regression.csv
Pickle file for tuned_logistic_regression successfully saved


#### Adjusted 4 K-Nearest Neighbour

In [125]:
def tune_KNN_model(X_train, y_train): 
    
    algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
    ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]
        
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')


    
    knn = KNeighborsClassifier()
    
    parameters = {'algorithm': algorithm,
                  'n_neighbors': ks
                     }
    tune = GridSearchCV(knn, parameters, scoring = scorer)
    tune.fit(X_train,y_train)
 
    
    return tune

In [126]:
clf = tune_KNN_model(X_train,y_train)
y_pred = clf.predict(X_val)

NameError: name 'tune_kn' is not defined

In [128]:
print(metrics.confusion_matrix(y_val,y_pred))

[[ 72 110  87   9]
 [ 45 274 101   5]
 [118 557 985  95]
 [ 60 104 182 360]]


In [129]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.24      0.26      0.25       278
           0       0.26      0.64      0.37       425
           1       0.73      0.56      0.63      1755
           2       0.77      0.51      0.61       706

    accuracy                           0.53      3164
   macro avg       0.50      0.49      0.47      3164
weighted avg       0.63      0.53      0.56      3164



In [130]:
accuracy_kn = metrics.accuracy_score(y_val,y_pred)
precision_kn = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_kn = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_kn = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save performance metrics
metrics_tuned_knn = {'accuracy': accuracy_kn,
                     'precision': precision_kn,
                     'recall': recall_kn,
                     'f1_score': f1_kn}

# show model performance
dict_to_df(metrics_tuned_knn)

,Value
accuracy,0.534450
precision,0.631157
recall,0.534450
f1_score,0.560240


In [131]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('tuned_k_nearest_neighbour', clf, X_test)

# save pickle file
make_pickle('tuned_k_nearest_neighbour', clf)

making prediction...
creating dataframe... 
Saved file submission_tuned_k_nearest_neighbour.csv
Pickle file for tuned_k_nearest_neighbour successfully saved


#### Adjusted 6 Random Forest

In [132]:
def tune_RF_model(X_train, y_train): 
    
    n_estimators = [10,20,30,50,100]
    max_depth = [1,5,10]
    max_features = ['auto', 'sqrt']   

    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')


    
    rf = RandomForestClassifier()
    
    parameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               
                 }
    
    tune = GridSearchCV(rf, parameters, scoring = scorer)
    tune.fit(X_train,y_train)
 
    
    return tune

In [135]:
clf = tune_RF_model(X_train,y_train)
y_pred = clf.predict(X_val)

In [35]:
model_name = 'resources/models/Logistic_regression.pkl'

model_save_path = 'resources/models/Logistic_regression.pkl'

# open file to write binary
with open(model_save_path,'wb') as file:

    # save model as file name
    pickle.dump(clf.best_estimator_,file)

In [137]:
print(metrics.classification_report(y_val,y_pred))

              precision    recall  f1-score   support

          -1       0.93      0.23      0.37       278
           0       0.61      0.32      0.42       425
           1       0.70      0.89      0.78      1755
           2       0.75      0.68      0.71       706

    accuracy                           0.71      3164
   macro avg       0.75      0.53      0.57      3164
weighted avg       0.72      0.71      0.68      3164



In [139]:
# Evaluate Model Performance
accuracy_rf = metrics.accuracy_score(y_val,y_pred)
precision_rf = metrics.precision_score(y_val,y_pred, average = 'weighted')
recall_rf = metrics.recall_score(y_val,y_pred, average = 'weighted')
f1_rf = metrics.f1_score(y_val,y_pred, average = 'weighted')

# save performance metrics
metrics_tuned_rf = {'accuracy': accuracy_rf,
                     'precision': precision_rf,
                     'recall': recall_rf,
                     'f1_score': f1_rf}

# show model performance
dict_to_df(metrics_tuned_rf)

,Value
accuracy,0.709545
precision,0.719845
recall,0.709545
f1_score,0.683602


In [140]:
# deploy model on comet
# deploy_comet(experiment, metrics_base_lsvc, lr.get_params())

# save to Kaggle
make_submission_csv('tuned_random_forest', clf, X_test)

# save pickle file
make_pickle('tuned_random_forest', clf)

making prediction...
creating dataframe... 
Saved file submission_tuned_random_forest.csv
Pickle file for tuned_random_forest successfully saved


<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/model-prediction.png" width = "50%" align = "left" />

Comparing Models

In [ ]:
acc = [accuracy_log,accuracy_sv, accuracy_nb, accuracy_kn, accuracy_rf]
f1 = [f1_log, f1_sv, f1_nb, f1_kn, f1_rf]
results = pd.DataFrame()



results['F1'] = f1
results['Accuracy'] = acc
results['models'] = ['Logistic Regression', 'Linear Support Vector', 'Naive Bayes', 'K Neighbours', 'Random Forest']
results.set_index('models', inplace= True)

Comparing All Models

In [ ]:
results_all = pd.DataFrame()

results_all['F1_base'] = f1_score
results_all['f1_tunned'] =f1
results_all['Accuracy_base'] = accuracy
results_all['Accuracy_tunned'] = acc

results_all['models'] = ['Logistic Regression', 'Linear Support Vector', 'Naive Bayes', 'K Neighbours', 'Random Forest']
results_all.set_index('models', inplace= True)

Building All Models (Stacked Models)

In [ ]:
names = ['Logistic Regression', 'Nearest Neighbors',' Naive Bayes',
         'Linear SVM',
          'Random Forest',  'AdaBoost']

classifiers = [
    MultinomialNB(alpha = 1),
    KNeighborsClassifier(n_neighbors =1),
    LinearSVC(C= 0.1, loss = 'squared_hinge'),
    RandomForestClassifier(),
    LogisticRegression(C=1 , penalty ='l2', random_state =1),
    AdaBoostClassifier()
]

In [ ]:
results = []

models = {}
confusion = {}
class_report = {}


for name, clf in zip(names, classifiers):
    print ('Fitting {:s} model...'.format(name))
    run_time = %timeit -q -o clf.fit(X_train, y_train)

    print ('... predicting')
    y_pred = clf.predict(X_train)
    y_pred_test = clf.predict(X_val)

    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred, average = 'weighted')
    recall    = metrics.recall_score(y_train, y_pred, average = 'weighted')

    f1        = metrics.f1_score(y_train, y_pred,average = 'weighted')
    f1_test   = metrics.f1_score(y_val, y_pred_test, average = 'weighted')

    # Save the results to dictionaries
    models[name] = clf
    confusion[name] = metrics.confusion_matrix(y_train, y_pred)
    class_report[name] = metrics.classification_report(y_train, y_pred)

    results.append([name, accuracy, precision, recall, f1, f1_test, run_time.best])


results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Classifier', inplace= True)

In [ ]:
results.sort_values('F1 Test', ascending=False)

## Experiment Results

Explain How experiment results will be presented

# Insights and Findings

<img src = "https://raw.githubusercontent.com/Mikentosh/ss5-classification/master/images/conclusion.png" width = "40%" align = "left" />